# 1. Import Library


In [1]:
import cv2
import numpy as np
import os
import time
import mediapipe as mp
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

# 2. Keypoints using MP Holistic

In [2]:
# Memanggil model mediapipe 
mp_holistic = mp.solutions.holistic         # Holistic model untuk mendeteksi landmark tubuh
mp_drawing = mp.solutions.drawing_utils     # Drawing utilities untuk menggambar landmark tubuh

In [3]:
# Function untuk mendeteksi dan menggambar landmark tubuh
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)      # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                       # Image is no longer writeable
    results = model.process(image)                      # Make prediction
    image.flags.writeable = True                        # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)      # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
# Function untuk menggambar landmark tubuh
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)      # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)          # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)     # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)    # Draw right hand connections

In [5]:
# Function untuk menggambar landmark tubuh dengan style edited 
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
# Mengakses webcam
cap = cv2.VideoCapture(0)

# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        
        # Read feed
        ret, frame = cap.read()
        
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# 3. Extract Keypoint Values

In [13]:
# Menaruh keypoint landmark ke dalam array
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [11]:
# Function untuk melakukan ekstraksi keypoint landmark dan menaruhnya ke dalam array
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [12]:
result_test = extract_keypoints(results)

In [ ]:
result_test.shape

# 4. Setup Folders for Collection

In [ ]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('D:/College/PKMKC/Sound Degla/Data') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [ ]:
# Membuat folder data per video untuk setiap action
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [ ]:
# Mengakses webcam
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                # print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                # Jika frame pertama
                if frame_num == 0: 
                    # Menampilkan text starting collection 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    
                # NEW Export keypoints
                # Melakukan ekstraksi keypoint landmark untuk satu frame
                keypoints = extract_keypoints(results)
                # Menyimpan ekstraksi keypoint landmark ke dalam file npy
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [ ]:
# Mengambil actions sebagai label dan mengubahnya menjadi numerical
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
# Mengambil keypoint landmark per frame setiap video untuk setiap action
sequences, labels = [], []      # Sequences -> fitur dari datanya
# Looping untuk setiap actions
for action in actions:
    # Looping untuk setiap video
    for sequence in range(no_sequences):
        window = []     # Window -> List untuk keypoint semua frame setiap video
        # Looping untuk setiap frame
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            # Memasukkan keypoint frame ke dalam list window
            window.append(res)
        # Memasukkan window ke list sequences per video
        sequences.append(window)
        # Menambah label untuk setiap action
        labels.append(label_map[action])

In [ ]:
# Menjadikan sequences sebagai fitur dari data
X = np.array(sequences)
X.shape

In [ ]:
# Mengubah label menjadi categorical list
y = to_categorical(labels).astype(int)
y[:10]

In [ ]:
# Memisahkan dataset menjadi data train & data test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# 7. Build and Train LSTM Neural Network

In [ ]:
# Menyimpan log callback dari proses training
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
# Membuat arsitektur model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))    # input_shape = (frame, keypoint)
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
# Melakukan compile pada model
model.compile(
    optimizer='Adam', 
    loss='categorical_crossentropy', 
    metrics=['categorical_accuracy']
)

In [ ]:
# Melakukan training
model.fit(
    X_train, 
    y_train, 
    epochs=2000, 
    callbacks=[tb_callback]
)

# 8. Make Predictions

In [ ]:
# Melakukan prediction dengan model yang telah dilatih
res = model.predict(X_test)

In [ ]:
# Menampilkan data prediksi pada index ke-5
actions[np.argmax(res[4])]

In [ ]:
# Menampilkan data true pada index ke-5
actions[np.argmax(y_test[4])]

# 9. Save Model

In [ ]:
# Menyimpan model yang telah ditraining
model.save('action.h5')

In [ ]:
# Menghapus model
del model

In [14]:
# Meload model yang telah ditraining
model = load_model('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [7]:
# Melakukan prediction dengan model yang telah diload
yhat = model.predict(X_test)

NameError: name 'X_test' is not defined

In [ ]:
# Mengubah data asli dan hasil prediksi menjadi list
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
# Menampilkan multilabel confusion matrix
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
# Menampilkan akurasi dari model
accuracy_score(ytrue, yhat)

# 11. Test in Real Time

In [15]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
# Function untuk menampilkan visualisasi probabilitas dari label
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [16]:
# 1. New detection variables
sequence = []       # List untuk setiap frame
sentence = []       # List untuk menaruh kata hasil prediksi
predictions = []    # List untuk menaruh hasil prediksi
threshold = 0.6     # Confidence matrix

# Mengakses webcam
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        
        # 2. Prediction logic
        # Mengekstrak keypoint
        keypoints = extract_keypoints(results)
        sequence.insert(0,keypoints)
        sequence = sequence[:30]
        # sequence.append(keypoints)
        # sequence = sequence[-30:]
        
        # Jika sudah 30 frame
        if len(sequence) == 30:
            # Melakukan prediction
            res = model.predict(np.expand_dims(sequence, axis=0))[0]    # np.expand_dims -> Berfungsi untuk mengubah shape pada array
            # Menampilkan hasil prediksi
            print(actions[np.argmax(res)])
            # Menambah hasil prediksi ke list predictions
            predictions.append(np.argmax(res))
            
            
        #3. Visualization logic
            # Mengecek hasil dari prediksi apakah sama dengan hasil prediksi pada 10 frame terakhir atau tidak
            if np.unique(predictions[-10:])[0]==np.argmax(res):
                # Mengecek apakah probabilitas hasil prediksi lebih dari threshold atau tidak
                if res[np.argmax(res)] > threshold: 
                    # Jika dalam sentence sudah ada kata
                    if len(sentence) > 0: 
                        # Mengecek action yang sedang dideteksi tidak sama dengan action yang terakhir 
                        if actions[np.argmax(res)] != sentence[-1]:
                            # Menambahkan kata ke dalam sentence
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            # Jika dalam sentence sudah ada 5 kata
            if len(sentence) > 5: 
                # Mengambil 5 kata yang terakhir dari sentence
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
        
        # Mengambar box di window
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        # Menaruh setence ke window
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 1s 1s/step


NameError: name 'actions' is not defined

: 

In [1]:
cap.release()
cv2.destroyAllWindows()

NameError: name 'cap' is not defined